In [ ]:
try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

from pathlib import Path
# Determine the locations of auxiliary libraries and datasets.
if IN_COLAB:
    google.colab.drive.mount("/content/drive")
    
    # Change this if you created the shortcut in a different location
    AUX_DATA_ROOT = Path("/content/drive/My Drive/Lenta Hack/hack_data")
    
    assert AUX_DATA_ROOT.is_dir(), "Have you forgot to 'Add a shortcut to Drive'?"
    
    import sys
    sys.path.insert(0, str(AUX_DATA_ROOT))
else:
    AUX_DATA_ROOT = Path(".")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
!pip install -U pyarrow

Requirement already up-to-date: pyarrow in /usr/local/lib/python3.6/dist-packages (1.0.1)


In [ ]:
data = pd.read_parquet(f"{AUX_DATA_ROOT}/transactions.parquet")
data = data.sample(frac=1)
data = data[:1000000]

In [ ]:
data

,chq_id,plant,chq_date,chq_position,client_id,material,sales_count,sales_sum,is_promo
12158371,89ff4b645a04852bd4e677e37385ea8d,b356b7a12680d1c1d12c729a28ed1079,2017-02-13,007,8dc137f73fa1896873231b02a950cfd1,077c19a0f9701a274909fdee5e73e811,1.000,32.89,1
27645148,df789004e9bd1ff65f7c03ce40c58d57,71f1c79e7e368a537e891c49552fa4c4,2017-08-12,010,999d99e1942464a7f064b0cf7d06b123,d245ec4770d83681f9b85b05c9458358,1.000,132.99,0
23570343,f0173774207a19a68e7b26fe320b8d60,028f7ee9eb0c0994a8ba1e60bcffd09e,2017-07-16,031,a1a4bab49af4c2495b0ea04561b23b05,4b897989447acebc51a7969ab4306e5a,0.712,199.35,0
20426434,ec14318b3d4db2787daf7522d969ee11,692bf3658e41106e0685c99ffc6f3343,2017-05-23,010,54e5b9f81ec23254c411ebe8df212b75,43d5f033b2abe9b156fea6f8b35d536c,2.000,59.98,0
28533960,22a5d9e517afeba2afee341aba9358d4,279f4fe9db63a9f4b87e18073956d7cf,2017-09-06,0000000004,6218e522b68cfa81631787d16313d366,5b8e6188b2cca50a0880e125ed835e09,2.000,139.78,1
...,...,...,...,...,...,...,...,...,...
31015651,e4cd31ac694c494639bc93f27357792f,a9b2b0ce7799c4de9c7a3aed9ebeddeb,2016-10-13,004,a81af0bd03ef70d68f5ecc383d05aab8,ab32d6b6cbb1ad526aaf3e22914644ec,0.504,184.41,0
24930307,83091a5181d760ace1636efbe7c0e899,a9b2b0ce7799c4de9c7a3aed9ebeddeb,2017-07-18,001,6e3cc87c78b3b00e1aa6a8eacf602d6f,45cf95ddee0ef5414c62c5c94f09c49f,1.000,39.90,1
25808245,08a7052d2758a6091c45cb0485c71dbd,279f4fe9db63a9f4b87e18073956d7cf,2017-08-01,009,d6e8dedb032d6c38ab97b619bf63f358,2c7f5905e701cacd8ba38b61bb71a10a,0.440,52.80,1
17916367,e5ea154d72ae6990846afb016ef9437c,f96816687580650f672ee11621c47dcb,2017-05-19,002,6ddd66e155b4fd4083dcc0a6e7552a66,95d544e73c7ffd5e3a156d00d730a55d,1.000,44.89,1


In [ ]:
data['price'] = data['sales_sum']/data['sales_count']
data

,chq_id,plant,chq_date,chq_position,client_id,material,sales_count,sales_sum,is_promo,price
12158371,89ff4b645a04852bd4e677e37385ea8d,b356b7a12680d1c1d12c729a28ed1079,2017-02-13,007,8dc137f73fa1896873231b02a950cfd1,077c19a0f9701a274909fdee5e73e811,1.000,32.89,1,32.890000
27645148,df789004e9bd1ff65f7c03ce40c58d57,71f1c79e7e368a537e891c49552fa4c4,2017-08-12,010,999d99e1942464a7f064b0cf7d06b123,d245ec4770d83681f9b85b05c9458358,1.000,132.99,0,132.990000
23570343,f0173774207a19a68e7b26fe320b8d60,028f7ee9eb0c0994a8ba1e60bcffd09e,2017-07-16,031,a1a4bab49af4c2495b0ea04561b23b05,4b897989447acebc51a7969ab4306e5a,0.712,199.35,0,279.985955
20426434,ec14318b3d4db2787daf7522d969ee11,692bf3658e41106e0685c99ffc6f3343,2017-05-23,010,54e5b9f81ec23254c411ebe8df212b75,43d5f033b2abe9b156fea6f8b35d536c,2.000,59.98,0,29.990000
28533960,22a5d9e517afeba2afee341aba9358d4,279f4fe9db63a9f4b87e18073956d7cf,2017-09-06,0000000004,6218e522b68cfa81631787d16313d366,5b8e6188b2cca50a0880e125ed835e09,2.000,139.78,1,69.890000
...,...,...,...,...,...,...,...,...,...,...
31015651,e4cd31ac694c494639bc93f27357792f,a9b2b0ce7799c4de9c7a3aed9ebeddeb,2016-10-13,004,a81af0bd03ef70d68f5ecc383d05aab8,ab32d6b6cbb1ad526aaf3e22914644ec,0.504,184.41,0,365.892857
24930307,83091a5181d760ace1636efbe7c0e899,a9b2b0ce7799c4de9c7a3aed9ebeddeb,2017-07-18,001,6e3cc87c78b3b00e1aa6a8eacf602d6f,45cf95ddee0ef5414c62c5c94f09c49f,1.000,39.90,1,39.900000
25808245,08a7052d2758a6091c45cb0485c71dbd,279f4fe9db63a9f4b87e18073956d7cf,2017-08-01,009,d6e8dedb032d6c38ab97b619bf63f358,2c7f5905e701cacd8ba38b61bb71a10a,0.440,52.80,1,120.000000
17916367,e5ea154d72ae6990846afb016ef9437c,f96816687580650f672ee11621c47dcb,2017-05-19,002,6ddd66e155b4fd4083dcc0a6e7552a66,95d544e73c7ffd5e3a156d00d730a55d,1.000,44.89,1,44.890000


In [ ]:
# clients_path = f"{AUX_DATA_ROOT}/clients.csv"
# clients = pd.read_csv(clients_path)

materials_path = f"{AUX_DATA_ROOT}/materials.csv"
materials = pd.read_csv(materials_path)

# plants_path = f"{AUX_DATA_ROOT}/plants.csv"
# plants = pd.read_csv(plants_path)

In [ ]:
materials

,material,hier_level_1,hier_level_2,hier_level_3,hier_level_4,vendor,is_private_label,is_alco
0,35cbdf61de9e19c8b417327aaef14c88,NONFOOD,ea5d2f1c4608232e07d3aa3d998e5135,99cad265a1768cc2dd013f0e740300ae,9eed45f71360b4b1e2590637467220e5,212a38db0ddcd009f1e164cc8483485c,0,0
1,c0b0bf24d4ec71da3d304f761ec555d8,NONFOOD,ea5d2f1c4608232e07d3aa3d998e5135,99cad265a1768cc2dd013f0e740300ae,de2eb747e0896c050905a6b635ab800a,9b0b6c7d55413ad3b67761b7b125b534,0,0
2,f0fc5e654a81a7c4b8ba8d7c26546e14,NONFOOD,ea5d2f1c4608232e07d3aa3d998e5135,99cad265a1768cc2dd013f0e740300ae,8a969031832c535daf96e0c2aed8e814,9b0b6c7d55413ad3b67761b7b125b534,0,0
3,353693e64fb5f9e2d29746d7fe6edf1e,NONFOOD,ea5d2f1c4608232e07d3aa3d998e5135,b58f7d184743106a8a66028b7a28937c,4c0dc012ebb679a18b244c53c6f59b5a,a3c8be149d718771e892619bd310b961,0,0
4,5d9e0b4302ce95448cae72165ff4cf5b,NONFOOD,ea5d2f1c4608232e07d3aa3d998e5135,b58f7d184743106a8a66028b7a28937c,4c0dc012ebb679a18b244c53c6f59b5a,a3c8be149d718771e892619bd310b961,0,0
...,...,...,...,...,...,...,...,...
105604,feeddbacd9eed3bac85b1571ac79e40e,NONFOOD,3295c76acbf4caaed33c36b1b5fc2cb1,11704817e347269b7254e744b5e22dac,e47e165040e2914783a451509bbc8180,8ca0d8745f91fb71551be26b1ca854f4,0,0
105605,e218c4c3676927e53b95a12ffe9e721a,FOOD,02e74f10e0327ad868d138f2b4fdd6f0,df0e09d6f25a15a815563df9827f48fa,a03ddfb4b641e98de3223765162217c3,e3319f2f4767af113d82174f662f2d4e,1,1
105606,9a5e2e34d8406616f5981868b4576444,NONFOOD,fc490ca45c00b1249bbe3554a4fdf6fb,865bf46435bd84fa5d89f64cf3ba7347,e2a75db437bee7d34615fccf32d7a464,546614030cdbddcae3de351852962759,0,0
105607,d208910eb82fa1f57684f793f93521a4,NONFOOD,ea5d2f1c4608232e07d3aa3d998e5135,90365351ccc7437a1309dc64e4db32a3,ea9cffc9bd2bf8a0fb128d223f6a4282,7f1dbcf6ab15c23093508627ca12b8b1,0,0


In [ ]:
data = pd.merge(data, materials, how = 'left', on='material')
data

,chq_id,plant,chq_date,chq_position,client_id,material,sales_count,sales_sum,is_promo,price,hier_level_1,hier_level_2,hier_level_3,hier_level_4,vendor,is_private_label,is_alco
0,89ff4b645a04852bd4e677e37385ea8d,b356b7a12680d1c1d12c729a28ed1079,2017-02-13,007,8dc137f73fa1896873231b02a950cfd1,077c19a0f9701a274909fdee5e73e811,1.000,32.89,1,32.890000,FOOD,17e62166fc8586dfa4d1bc0e1742c08b,b90ba83119860d7f6a6dfaab9f2aa150,9b265fdfc7880e370c421b7f1c1f2d70,9402a4e22fab4614e613d15e63f57f37,0.0,0.0
1,df789004e9bd1ff65f7c03ce40c58d57,71f1c79e7e368a537e891c49552fa4c4,2017-08-12,010,999d99e1942464a7f064b0cf7d06b123,d245ec4770d83681f9b85b05c9458358,1.000,132.99,0,132.990000,FOOD,34173cb38f07f89ddbebc2ac9128303f,5f268dfb0fbef44de0f668a022707b86,95d8587c3cbd6b20de81573fac33d24c,c3db4bd6f9f4961f43e448f55a0f2395,0.0,0.0
2,f0173774207a19a68e7b26fe320b8d60,028f7ee9eb0c0994a8ba1e60bcffd09e,2017-07-16,031,a1a4bab49af4c2495b0ea04561b23b05,4b897989447acebc51a7969ab4306e5a,0.712,199.35,0,279.985955,FOOD,1ff1de774005f8da13f42943881c655f,4dd9cec1c21bc54eecb53786a2c5fa09,ac3a5d655b2ddfb54e97991fc12644a0,a90f6baea213548828f36a3652d115df,0.0,0.0
3,ec14318b3d4db2787daf7522d969ee11,692bf3658e41106e0685c99ffc6f3343,2017-05-23,010,54e5b9f81ec23254c411ebe8df212b75,43d5f033b2abe9b156fea6f8b35d536c,2.000,59.98,0,29.990000,NONFOOD,d67d8ab4f4c10bf22aa353e27879133c,63154d5661f774fb7d2d11701d466aa2,b73679d1380df653cb743962af76e11d,35c694fe8e964fec08924e5cbeb12bdd,1.0,0.0
4,22a5d9e517afeba2afee341aba9358d4,279f4fe9db63a9f4b87e18073956d7cf,2017-09-06,0000000004,6218e522b68cfa81631787d16313d366,5b8e6188b2cca50a0880e125ed835e09,2.000,139.78,1,69.890000,FOOD,182be0c5cdcd5072bb1864cdee4d3d6e,f24ad6f72d6cc4cb51464f2b29ab69d3,8b87d89c20466be32b28ceab1fb99830,0d146fab468bb86c66db04ce17d6a3ae,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,e4cd31ac694c494639bc93f27357792f,a9b2b0ce7799c4de9c7a3aed9ebeddeb,2016-10-13,004,a81af0bd03ef70d68f5ecc383d05aab8,ab32d6b6cbb1ad526aaf3e22914644ec,0.504,184.41,0,365.892857,FOOD,6512bd43d9caa6e02c990b0a82652dca,4da04049a062f5adfe81b67dd755cecc,5ee2184526dad0f8bc6056fa2438c588,a3d2de7675556553a5f08e4c88d2c228,0.0,0.0
999996,83091a5181d760ace1636efbe7c0e899,a9b2b0ce7799c4de9c7a3aed9ebeddeb,2017-07-18,001,6e3cc87c78b3b00e1aa6a8eacf602d6f,45cf95ddee0ef5414c62c5c94f09c49f,1.000,39.90,1,39.900000,FOOD,c74d97b01eae257e44aa9d5bade97baf,1bc0249a6412ef49b07fe6f62e6dc8de,60787c41b04097dfea76addfccd12243,a3d2de7675556553a5f08e4c88d2c228,0.0,0.0
999997,08a7052d2758a6091c45cb0485c71dbd,279f4fe9db63a9f4b87e18073956d7cf,2017-08-01,009,d6e8dedb032d6c38ab97b619bf63f358,2c7f5905e701cacd8ba38b61bb71a10a,0.440,52.80,1,120.000000,FOOD,6364d3f0f495b6ab9dcf8d3b5c6e0b01,d77e68596c15c53c2a33ad143739902d,b4de02e926a96b64e43a0b491457cf70,d9a58e5502f779d7e17a1505689ed1b8,0.0,0.0
999998,e5ea154d72ae6990846afb016ef9437c,f96816687580650f672ee11621c47dcb,2017-05-19,002,6ddd66e155b4fd4083dcc0a6e7552a66,95d544e73c7ffd5e3a156d00d730a55d,1.000,44.89,1,44.890000,FOOD,33e75ff09dd601bbe69f351039152189,2eb5657d37f474e4c4cf01e4882b8962,e2fb583d6b3828b6ae32615c3f540f70,a6945e42a5360a2dd4bfcdd5c002e2aa,0.0,1.0


In [ ]:
freq = data.groupby(['client_id', 'hier_level_3', 'hier_level_4', 'material']).chq_id.count().reset_index().sort_values('chq_id',ascending=False).set_index('client_id')
freq.rename(columns = {"chq_id":"frequency"}, inplace = True)

In [ ]:
freq.head(40)

,hier_level_3,hier_level_4,material,frequency
client_id,,,,
35e38d87f20cead704467ba83c0f0ec3,7501e5d4da87ac39d782741cd794002d,9076db4895b417d4a668cb0b7168e4db,c8ac520562ba934dd67cbf0f060c4252,38
dfb684ff08c6dea880d5eb9f0e9309cf,a3788c8c64fd65c470e23e7534c3ebc8,29a90297dcc11ab043a1ffb971246c08,c6571b7b1b7ce43041791bd91a5a65dd,36
d316c6fb1ecfc3303ef286ad7426a642,7501e5d4da87ac39d782741cd794002d,9076db4895b417d4a668cb0b7168e4db,c8ac520562ba934dd67cbf0f060c4252,30
0b1b96c791f5779ad006171187b2d31e,7501e5d4da87ac39d782741cd794002d,9076db4895b417d4a668cb0b7168e4db,c8ac520562ba934dd67cbf0f060c4252,23
ac50f6d558766dceed03dae195d0d151,2eb5657d37f474e4c4cf01e4882b8962,007496ff404d0a1d4e5c993492d4b99a,6964e7ca85a4815c0c9dd8255c7bcff4,22
423de53eb432daa7273f65b1ef4368f0,7501e5d4da87ac39d782741cd794002d,9076db4895b417d4a668cb0b7168e4db,c8ac520562ba934dd67cbf0f060c4252,21
9e486ace04de5e3b2529d76d115dcbba,7501e5d4da87ac39d782741cd794002d,9076db4895b417d4a668cb0b7168e4db,c8ac520562ba934dd67cbf0f060c4252,19
ef0bb7c50945fd32967badc6f7e3134e,7d265aa7147bd3913fb84c7963a209d1,65ce9a05680f7ff3339a68cc1eac2330,82db77b19a1d3b622fedf5b1f7dbc1ed,18
cd2a8f57759a0d247db058321b6a12b3,a3788c8c64fd65c470e23e7534c3ebc8,29a90297dcc11ab043a1ffb971246c08,d99ff1bb6ab63242de3b682b51461dce,18


In [ ]:
promo = data.groupby(['client_id', 'hier_level_3', 'hier_level_4', 'material']).agg({'is_promo': 'sum'})
promo

is_promo
client_id                        hier_level_3                     hier_level_4                     material                                  
0000d48a8891cd462fdf01ea861ec53e 0c7119e3a6a2209da6a5b90e5b5b75bd e24fc73cf4d574c17eeb75682c0c39cf 441f83db20dab8cbdd080b53307836e2         0
                                 0c8ce55163055c4da50a81e0a273468c 8786e661759e997352c87daea839f78c be2296a9c69d93ec7d9f97cadeafadb6         1
                                 3fab5890d8113d0b5a4178201dc842ad c4403794ed3270a96121683b9e7bca97 b477e1afeca3472839e93b56df1a54de         0
                                 4e0223a87610176ef0d24ef6d2dcde3a 7b6ca9e5defffac176ccce54d800d4c6 8c5779283317adc89829cf31803442a1         0
                                 6211080fa89981f66b1a0c9d55c61d0f cf336c4236916272f4f49ec07388fdc4 c88bad1e43bf3c913f64528ae07c4495         1
...                                                                                                                                       ...
fffc3f2be1422d248dbeda4196ef0711 c559da2ba967eb820766939a658022c8 34d2777e78ccf007523e2dd1fcea6b2d 8b682bcb4f749727567ba031cd7fcc36         0
fffc477269ae5f219e39aa51fa7abb81 5505712229fb1eb500efadddc0353264 0079c916e4207b3b87ba54cd0565ecc0 a27438baa42201e1bb9be8c0d7c2bc75         1
                                 996009f2374006606f4c0b0fda878af1 0aac5ea0d2a927322dd4c09ef37c0be0 581a6fa533cc0c71c0f139be5d7b474c         0
                                 e71e5cd119bbc5797164fb0cd7fd94a4 91534231d73afa8f5765e15d83711a52 6c54ca6fd28928b73e9b6c3e796c68b7         1
fffd3e78e3ea6bf00aee1372efc09dad 2a3d6d6cc4b5e77238c1fc1bb6cdd681 f1471bc23e88cb9d971953f99e020188 be497986bd5afd60acc5b5b0e7bdfcee         0

[948794 rows x 1 columns]

In [ ]:
freq_promo = pd.merge(freq, promo, how = 'left', on=['client_id', 'hier_level_3', 'hier_level_4', 'material'])
freq_promo['price'] = freq_promo[['material']].apply(lambda x : data.query('material == "%s"' % x[0]).price.mean(), axis=1)

In [ ]:
freq_promo.reset_index(inplace=True)

## Customer Categorization

In [ ]:
def is_article_oriented(row):
  client = row.client_id
  material = row.material
  freq = row.frequency
  price = row.price
  level = row.hier_level_4

  discounts = True
  cheap = True

  client_table = freq_promo.loc[((freq_promo.client_id == client) & (freq_promo.material == material))]

  if (len(client_table[client_table.is_promo == 0]) > 0): # The client buys the article without a discount even for one time
    discounts = False
  
  materials_table = freq_promo[(freq_promo.hier_level_4 == level)]
  if (materials_table.price.mean() < price): # The client buys the most expensive material
    cheap = False

  return not (discounts or cheap)


freq_promo['is_article_oriented'] = freq_promo.apply(is_article_oriented, axis=1)
freq_promo.head()

In [ ]:
print(freq_promo[freq_promo.is_article_oriented == True].shape[0] / freq_promo.shape[0])

## Article Scoring

In [ ]:
def has_negative_feedback(client, material):
  q = data.query('client_id == "%s" and material == "%s"' % (client, material))
  return len(q[q.return_count >= 1]) > 0  #Has a negative feedback
def compute_score(row):
  client = row.client_id
  material = row.material
  level = row.hier_level_4
  freq = row.frequency
  score = 0
  if not has_negative_feedback(client, material):
    score = freq / freq_promo.query('client_id == "%s" and hier_level_4 == "%s"' % (client, level)).frequency.sum()
  
  return score

freq_promo['material_score'] = freq_promo.apply(compute_score, axis=1)
freq_promo.head()

In [ ]:
freq_promo[freq_promo.material_score < 1]

In [ ]:
freq_promo[((freq_promo.hier_level_4 == '65ce9a05680f7ff3339a68cc1eac2330') & (freq_promo.client_id == '759165e34fbdc27550ccf84ceeefac17'))]

## Data split

In [ ]:
price_oriented_people = freq_promo[~ freq_promo.is_article_oriented].copy()
article_oriented_people = freq_promo[freq_promo.is_article_oriented].copy()

In [ ]:
price_oriented_people.head()

In [ ]:
article_oriented_people.head()

#AO Score vs. PO Score (Article Oriented Vs. product Oriented)


In [ ]:
def compute_total_score(row, po=True):
  material = row.material
  if po:
   return price_oriented_people[price_oriented_people.material == material].material_score.sum()

  else:
    return article_oriented_people[article_oriented_people.material == material].material_score.sum()


In [ ]:
price_oriented_people['PO_Score'] = price_oriented_people.apply(compute_total_score, axis=1)

In [ ]:
price_oriented_people.head()

In [ ]:
article_oriented_people['AO_Score'] = article_oriented_people[['material']].apply(compute_total_score, axis=1, args=(False,))

In [ ]:
article_oriented_people.head()

# FIND PRICES FOR MATERIALS 

In [ ]:
df1 = data[['client_id', 'price', 'hier_level_3', 'hier_level_4', 'material', 'sales_count']].copy()
def compute_count(x):
  if x.sales_count < 0:
    return 1
  else:
    return 0
df1['return_count'] = df1.apply(compute_count, axis=1)
df1 = df1.groupby(['hier_level_3', 'hier_level_4', 'material']).agg({'price': 'mean', 'return_count': 'sum'} , axis = 0).reset_index()
df1

In [ ]:
df1['material'].count()

In [ ]:
data.drop('price' , axis = 1, inplace = True, errors='ignore')

In [ ]:
def price_for_material(x):
  return x.price
data['price'] = df1.apply(price_for_material, axis = 1)
data

In [ ]:
df2 = data[['client_id', 'price', 'hier_level_3', 'hier_level_4', 'material', 'sales_count']].copy()
def compute_count(x):
  if x.sales_count < 0:
    return 1
  else:
    return 0
df2['return_count'] = df2.apply(compute_count, axis=1)
df2 = df2.groupby(['client_id', 'hier_level_3', 'hier_level_4', 'material']).agg({'price': 'mean', 'return_count': 'sum'} , axis = 0).reset_index()
df2

# FINAL TABLE

# FREQ_PROMO_PRICE

In [ ]:
# freq_promo_price = pd.merge(freq_promo, df1, how = 'left', on=['client_id','hier_level_3', 'hier_level_4','material'])
# freq_promo_price = freq_promo_price.drop('sales_count', axis = 1)
# freq_promo_price

In [ ]:
#freq_promo_price.head(40)